In [1]:
!pip install openai jsonlines

In [22]:
import shutil, os, subprocess

import json
import time
from pprint import pprint

from openai import OpenAI
import jsonlines
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/tomoro_convfinqa_finetuning/dataset_2/"

OPENAI_API_KEY = "<replace with your openai api key>"

Mounted at /content/drive


In [4]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
with jsonlines.open(DATA_DIR + "final_train_set.jsonl") as file:
    data = list(file.iter())

pprint(data)

Streaming output truncated to the last 5000 lines.
                           'losses on investments and could have a material '
                           'adverse impact on our results of operations , '
                           'equity , business and insurer financial strength '
                           'and debt ratings . our results could be adversely '
                           'affected by catastrophic events . we are exposed '
                           'to unpredictable catastrophic events , including '
                           'weather-related and other natural catastrophes , '
                           'as well as acts of terrorism . any material '
                           'reduction in our operating results caused by the '
                           'occurrence of one or more catastrophes could '
                           'inhibit our ability to pay dividends or to meet '
                           'our interest and principal payment obligations . '
              

# Upload training file

In [ ]:
result_file = client.files.create(
  file=open(DATA_DIR + "final_train_set.jsonl", "rb"),
  purpose="fine-tune"
)

pprint(result_file)

FileObject(id='file-CYnsiD621jtPKnuTzfCMqu', bytes=4521598, created_at=1751210820, filename='final_train_set.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


In [ ]:
param_training_file_name = result_file.id
pprint(param_training_file_name)

'file-CYnsiD621jtPKnuTzfCMqu'


# Create fine-tuning job

In [ ]:
result_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4.1-mini-2025-04-14"
)

pprint(result_job)

FineTuningJob(id='ftjob-c0hQ4BDSmOAuZWTDAGxEg90o', created_at=1751210830, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-mini-2025-04-14', object='fine_tuning.job', organization_id='org-n1sAjiCyXrZLVcPqfbptX3rN', result_files=[], seed=243762131, status='validating_files', trained_tokens=None, training_file='file-CYnsiD621jtPKnuTzfCMqu', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)


In [ ]:
param_file_tune_job_id = result_job.id
pprint(param_file_tune_job_id)

'ftjob-c0hQ4BDSmOAuZWTDAGxEg90o'


# Retrieve the status of a fine-tuning job

In [ ]:
# Retrieve state
client.fine_tuning.jobs.retrieve(param_file_tune_job_id)

FineTuningJob(id='ftjob-c0hQ4BDSmOAuZWTDAGxEg90o', created_at=1751210830, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4.1-mini-2025-04-14:tsgs::BnoieExO', finished_at=1751212735, hyperparameters=Hyperparameters(batch_size=2, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-4.1-mini-2025-04-14', object='fine_tuning.job', organization_id='org-n1sAjiCyXrZLVcPqfbptX3rN', result_files=['file-Jj7u1DA9BYxHJM1GvLDm17'], seed=243762131, status='succeeded', trained_tokens=2969043, training_file='file-CYnsiD621jtPKnuTzfCMqu', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=2, learning_rate_multiplier=2.0, n_epochs=3))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [ ]:
# Retrieve status
client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status

'succeeded'

In [ ]:
param_file_tune_model = client.fine_tuning.jobs.retrieve(param_file_tune_job_id).fine_tuned_model

# Test model for inference

In [5]:
with jsonlines.open(DATA_DIR + "final_test_set.jsonl") as file:
    eval_data = list(file.iter())

In [30]:
test_question_idx = random.randint(0, 199)
print(test_question_idx)

74


In [31]:
eval_data[test_question_idx]['messages']

[{'role': 'system',
  'content': 'management 2019s discussion and analysis 128 jpmorgan chase & co./2010 annual report year ended december 31 .\n\nTABLE:\n | hedges of lending-related commitments ( a ) | cva and hedges of cva ( a ) | net gains/ ( losses )\n--- | --- | --- | ---\n2010 | -279.0 | -403.0 | -682.0\n2009 | -3258.0 | 1920.0 | -1338.0\n2008 | 2216.0 | -2359.0 | -143.0\n\n( a ) these hedges do not qualify for hedge accounting under u.s . gaap . lending-related commitments jpmorgan chase uses lending-related financial instruments , such as commitments and guarantees , to meet the financing needs of its customers . the contractual amount of these financial instruments represents the maximum possible credit risk should the counterpar- ties draw down on these commitments or the firm fulfills its obliga- tion under these guarantees , and should the counterparties subsequently fail to perform according to the terms of these con- tracts . wholesale lending-related commitments were $ 

In [32]:
test_question = eval_data[test_question_idx]['messages'][0:2]

In [33]:
completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=test_question
)
turn_1_answer = completion.choices[0].message.content
pprint(turn_1_answer)

test_question.append({'role': 'assistant', 'content': turn_1_answer})
test_question.append(eval_data[test_question_idx]['messages'][3])     #append turn_2_question

'346.1'


In [34]:
completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=test_question
)
pprint(completion.choices[0].message.content)

'347.2'
